<a href="https://colab.research.google.com/github/saliq-2/NN-A3-assignment/blob/main/7IntroductiontoHuggingfaceTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets evaluate sacrebleu --quiet
!pip install --upgrade -q transformers


In [ ]:
import os
import random
import numpy as np
import pandas as pd

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import evaluate


2025-07-30 13:42:48.645868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753882968.675976     208 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753882968.684555     208 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from datasets import Dataset

# Load EN and DE text files
with open("/kaggle/input/europarl-train-data/dataset.train.en", encoding="ISO-8859-1") as f:
    en_lines = f.read().strip().split("\n")

with open("/kaggle/input/europarl-train-data/dataset.train.de", encoding="ISO-8859-1") as f:
    de_lines = f.read().strip().split("\n")

# Ensuring equal length
min_len = min(len(en_lines), len(de_lines))
en_lines, de_lines = en_lines[:min_len], de_lines[:min_len]

# Prepare dataset format
data = [{"translation": {"en": en, "de": de}} for en, de in zip(en_lines, de_lines)]

# Create Hugging Face dataset and split
raw_dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

print(raw_dataset)


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 201
    })
})


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-de"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

source_lang = "en"
target_lang = "de"
prefix = ""

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3950: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/201 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 16

# Safe version of training arguments for older transformers
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [ ]:
batch_size = 16

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [ ]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Evaluate before training
initial_eval = trainer.evaluate()
print(" Evaluation Before Training:")
print(initial_eval)


/tmp/ipykernel_208/2186300128.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


📊 Evaluation Before Training:
{'eval_loss': 7.538430213928223, 'eval_model_preparation_time': 0.0036, 'eval_bleu': 0.0157, 'eval_gen_len': 290.8806, 'eval_runtime': 3474.3963, 'eval_samples_per_second': 0.058, 'eval_steps_per_second': 0.004}


In [ ]:
trainer.train()


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3852: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=113, training_loss=5.095050102841538, metrics={'train_runtime': 1098.1979, 'train_samples_per_second': 1.639, 'train_steps_per_second': 0.103, 'total_flos': 61017056870400.0, 'train_loss': 5.095050102841538, 'epoch': 1.0})

In [ ]:
final_eval = trainer.evaluate()
print("Evaluation After Training:")
print(final_eval)


📊 Evaluation After Training:
{'eval_loss': 4.041402816772461, 'eval_model_preparation_time': 0.0036, 'eval_bleu': 0.2878, 'eval_gen_len': 15.6368, 'eval_runtime': 775.2275, 'eval_samples_per_second': 0.259, 'eval_steps_per_second': 0.017, 'epoch': 1.0}
